In [1]:
#print("Hello 10")
inputs  = [1, 2, 3, 2.5]
weights = [ [0.2, 0.8, -0.5, 1],
            [0.5, -0.91, 0.26, -0.5],
            [-0.26, -0.27, 0.17, 0.87]]
biases = [2, 3, 0.5]



layer_outputs = []

#iterate each neuron and -> creates tupe of weights and bias
for neuron_weights, neuron_bias in zip(weights, biases):
    neuron_output = 0
    #iterate each input
    for n_input, weight in zip(inputs, neuron_weights):
        #sum the inputs and weights for the neuron
        neuron_output += n_input*weight
    #add the bias
    neuron_output += neuron_bias
    layer_outputs.append(neuron_output)
print(layer_outputs)



[4.8, 1.21, 2.385]


In [2]:
test = zip(weights, biases)
print(tuple(test))


(([0.2, 0.8, -0.5, 1], 2), ([0.5, -0.91, 0.26, -0.5], 3), ([-0.26, -0.27, 0.17, 0.87], 0.5))


In [3]:
#tensors, arrays and vectors
#list
l = [1,5,6,2]
lol = [[1,5,6,2],
       [3,2,1,3]]
lolol = [[[1,5,6,2],
       [3,2,1,3]],
         [[1,5,6,2],
       [3,2,1,3]],
         [[1,5,6,2],
       [3,2,1,3]] ]



In [15]:
import numpy as np
#dot products and vector additios
inputs  = [1, 2, 3, 2.5]
weights = [ [0.2, 0.8, -0.5, 1],
            [0.5, -0.91, 0.26, -0.5],
            [-0.26, -0.27, 0.17, 0.87]]
biases = [2, 3, 0.5]

outputs = np.dot(weights,inputs) +biases
print(outputs)

[4.8   1.21  2.385]
